In [9]:
%pylab
from time import sleep
from collections import deque
from serial import Serial

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
# Config
PORT = "/dev/ttyACM0"
SAMPLE_SIZE = 600
BAUD_RATE = 115200
TIME_STEP = 0.01

In [11]:
arduino = None

def getData():
    global arduino
    
    if not arduino:
        arduino = Serial(PORT, BAUD_RATE, timeout=0.1)
        print("Opened", arduino.name)
        sleep(3)
        arduino.readline() # Flush input

    ser_bytes = arduino.readline()
    decoded_bytes = ser_bytes[0:len(ser_bytes)-2].decode("utf-8", errors='ignore') # remove trailing characters (\r\n)
    
    if not "Data:" in decoded_bytes:
        return None
    
    vals = decoded_bytes.replace("Data:", "").strip().split(',')

    if len(vals) != 6:
        return None

    vals = [float(i) for i in vals]

    return vals

for _ in range(10):
    print(getData())

Opened /dev/ttyACM0
[0.0021, 0.0001, 0.1227, 0.1575, -0.0481, -0.1619]
None
[0.002, -0.0001, 0.1232, 0.14, -0.0569, -0.1575]
[0.0024, 0.0002, 0.1231, 0.1356, -0.0481, -0.1619]
[0.002, 0.0002, 0.1231, 0.14, -0.0613, -0.1575]
[0.0018, 0.0002, 0.1232, 0.1444, -0.0481, -0.1619]
[0.0023, 0.0007, 0.1233, 0.1488, -0.0481, -0.1619]
[0.0023, 0.0003, 0.1226, 0.1356, -0.0438, -0.1575]
[0.0019, 0.0001, 0.1229, 0.1356, -0.0525, -0.1619]
[0.0015, -0.0003, 0.1239, 0.14, -0.0481, -0.1706]


# Gyroscope offset calibration

In [16]:
print("Put down the board and do not touch or move it!")

for seconds in range(3, 0, -1):
    print(seconds, end='...')
    time.sleep(1)
    
print("\nCOLLECTING IMU DATA")

# close port in case its open
if arduino:
    arduino.close()
    arduino = None

previos_x_gyro = 0
previos_y_gyro = 0
previos_z_gyro = 0

# Deque for axes
gyro_x = deque(maxlen=SAMPLE_SIZE)
gyro_y = deque(maxlen=SAMPLE_SIZE)
gyro_z = deque(maxlen=SAMPLE_SIZE)

for _ in range(SAMPLE_SIZE):
    data = getData()
    if data is not None:
        
        x_gyro, y_gyro, z_gyro = data[3:6]          

        if(abs(x_gyro) < abs(3 * previos_x_gyro) 
           and abs(y_gyro) < abs(3 * previos_y_gyro) and abs(z_gyro) < abs(3 * previos_z_gyro)):
            gyro_x.append(x_gyro)
            gyro_y.append(y_gyro)
            gyro_z.append(z_gyro)
        
        previos_x_gyro, previos_y_gyro, previos_z_gyro = x_gyro, y_gyro, z_gyro
            
            
gyro_min_x = min(gyro_x)
gyro_max_x = max(gyro_x)
gyro_min_y = min(gyro_y)
gyro_max_y = max(gyro_y)
gyro_min_z = min(gyro_z)
gyro_max_z = max(gyro_z)

print("Gyro X range: ", gyro_min_x, gyro_max_x)
print("Gyro Y range: ", gyro_min_y, gyro_max_y)
print("Gyro Z range: ", gyro_min_z, gyro_max_z)

gyro_calibration = [ (gyro_max_x + gyro_min_x) / 2, (gyro_max_y + gyro_min_y) / 2, (gyro_max_z + gyro_min_z) / 2]
print("Final calibration in deg/s:", gyro_calibration)

fig, (uncal, cal) = plt.subplots(2, 1)

# Clear all axis
uncal.cla()
t = np.linspace(0, len(gyro_x), len(gyro_x))

# plot uncalibrated data
uncal.plot(t, gyro_x, color='r', label='x')
uncal.plot(t, gyro_y, color='g', label='y')
uncal.plot(t, gyro_z, color='b', label='z')
uncal.title.set_text("Uncalibrated Gyro")
uncal.set(ylabel='Deg/s')
uncal.set(xlabel='Samples')
uncal.legend(loc="upper left")

# plot calibrated data
cal.plot(t, [x - gyro_calibration[0] for x in gyro_x], color='r', label='x')
cal.plot(t, [y - gyro_calibration[1] for y in gyro_y], color='g', label='y')
cal.plot(t, [z - gyro_calibration[2] for z in gyro_z], color='b', label='z')
cal.title.set_text("Calibrated Gyro")
cal.set(ylabel='Deg/s')
cal.set(xlabel='Samples')
cal.legend(loc="upper left")

fig.tight_layout()
fig.show()

Put down the board and do not touch or move it!
3...2...1...
COLLECTING IMU DATA
Opened /dev/ttyACM0
Gyro X range:  -0.0219 0.0175
Gyro Y range:  -0.0287 0.0194
Gyro Z range:  -0.0094 0.0169
Final calibration in deg/s: [-0.002199999999999999, -0.00465, 0.003749999999999999]


In [23]:
print("Put down the board and do not touch or move it!")

for seconds in range(3, 0, -1):
    print(seconds, end='...')
    time.sleep(1)
    
print("\nCOLLECTING IMU DATA")

# close port in case its open
if arduino:
    arduino.close()
    arduino = None

previos_x_accel = 0
previos_y_accel = 0
previos_z_accel = 0

# Deque for axes
accel_x = deque(maxlen=SAMPLE_SIZE)
accel_y = deque(maxlen=SAMPLE_SIZE)
accel_z = deque(maxlen=SAMPLE_SIZE)
for _ in range(SAMPLE_SIZE):
    data = getData()
    if data is not None:
        
        x_accel, y_accel, z_accel = data[0:3]
        
        if(abs(x_accel) < abs(5 * previos_x_accel) 
           and abs(y_accel) < abs(5 * previos_y_accel) and abs(z_accel) < abs(5 * previos_z_accel)):
            accel_x.append(x_accel)
            accel_y.append(y_accel)
            accel_z.append(z_accel)
        
        previos_x_accel, previos_y_accel, previos_z_accel = x_accel, y_accel, z_accel
            
            
accel_min_x = min(accel_x)
accel_max_x = max(accel_x)
accel_min_y = min(accel_y)
accel_max_y = max(accel_y)
accel_min_z = min(accel_z)
accel_max_z = max(accel_z)

print("Accel X range: ", accel_min_x, accel_max_x)
print("Accel Y range: ", accel_min_y, accel_max_y)
print("Accel Z range: ", accel_max_z, accel_max_z)

accel_calibration = [ (accel_max_x + accel_min_x) / 2, (accel_max_y + accel_min_y) / 2, (accel_max_z + accel_min_z) / 2]
print("Final calibration in g's:", accel_calibration)

fig, (uncal, cal) = plt.subplots(2, 1)

# Clear all axis
uncal.cla()
t = np.linspace(0, len(accel_x), len(accel_x))

# plot uncalibrated data
uncal.plot(t, accel_x, color='r', label='x')
uncal.plot(t, accel_y, color='g', label='y')
uncal.plot(t, accel_z, color='b', label='z')
uncal.title.set_text("Uncalibrated Accelerometer")
uncal.set(ylabel='g')
uncal.set(xlabel='Samples')
uncal.legend(loc="upper left")

# plot calibrated data
cal.plot(t, [x - accel_calibration[0] for x in accel_x], color='r', label='x')
cal.plot(t, [y - accel_calibration[1] for y in accel_y], color='g', label='y')
cal.plot(t, [z - accel_calibration[2] for z in accel_z], color='b', label='z')
cal.title.set_text("Calibrated Accelerometer")
cal.set(ylabel='g')
cal.set(xlabel='Samples')
cal.legend(loc="upper left")

fig.tight_layout()
fig.show()
arduino.close()

Put down the board and do not touch or move it!
3...2...1...
COLLECTING IMU DATA
Opened /dev/ttyACM0
Accel X range:  -0.001 0.001
Accel Y range:  -0.0049 0.0113
Accel Z range:  0.0011 0.0011
Final calibration in g's: [0.0, 0.0031999999999999997, 0.00015000000000000001]
